# Segmenting and Clustering Neighborhoods in Toronto  3

In [1]:
!pip install geopy
!pip install folium
print("Installed!")

     |████████████████████████████████| 94 kB 6.4 MB/s  eta 0:00:01
Installed!


In [2]:
import folium
import requests 
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("Imported!")

Imported!


In [3]:
df = pd.read_csv('toronto_part2.csv')
print(df.shape)
df.head()

(20, 5)


,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,M9AEtobicoke(Islington Avenue),43.64869,-79.38544
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,M9BEtobicoke(West Deane Park / Princess Garden...,43.81023,-79.22038
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...,43.78716,-79.13252
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,M9ENot assigned,43.74953,-79.18992
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,M9GNot assigned,43.76748,-79.22829


In [6]:
!conda install -c conda-forge geocoder --yes
import geocoder
from geopy.geocoders import Nominatim 

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.12.0             |     pyhd8ed1ab_0          96 KB  conda-forge
    aiohttp-3.7.4              |   py37h5e8e339_0      

xlrd-2.0.1           | 92 KB     | ##################################### | 100% 
argon2-cffi-20.1.0   | 47 KB     | ##################################### | 100% 
giflib-5.2.1         | 77 KB     | ##################################### | 100% 
ca-certificates-2020 | 137 KB    | ##################################### | 100% 
click-7.1.2          | 64 KB     | ##################################### | 100% 
cached_property-1.5. | 11 KB     | ##################################### | 100% 
pyasn1-0.4.8         | 53 KB     | ##################################### | 100% 
jupyter_client-6.1.1 | 79 KB     | ##################################### | 100% 
blinker-1.4          | 13 KB     | ##################################### | 100% 
numpy-1.20.2         | 5.8 MB    | ##################################### | 100% 
libxml2-2.9.10       | 1.3 MB    | ##################################### | 100% 
libffi-3.3           | 51 KB     | ##################################### | 100% 
webencodings-0.5.1   | 12 KB

liblapack-3.9.0      | 11 KB     | ##################################### | 100% 
plotly-4.14.3        | 5.9 MB    | ##################################### | 100% 
libclang-11.1.0      | 19.2 MB   | ##################################### | 100% 
pickleshare-0.7.5    | 9 KB      | ##################################### | 100% 
openssl-1.1.1k       | 2.1 MB    | ##################################### | 100% 
biopython-1.78       | 2.6 MB    | ##################################### | 100% 
nbformat-5.1.2       | 66 KB     | ##################################### | 100% 
locket-0.2.0         | 6 KB      | ##################################### | 100% 
lzo-2.10             | 314 KB    | ##################################### | 100% 
fontconfig-2.13.1    | 344 KB    | ##################################### | 100% 
lz4-c-1.9.3          | 179 KB    | ##################################### | 100% 
importlib-metadata-3 | 27 KB     | ##################################### | 100% 
pyodbc-4.0.30        | 71 KB

libglib-2.68.0       | 3.0 MB    | ##################################### | 100% 
terminado-0.9.4      | 26 KB     | ##################################### | 100% 
keyring-18.0.0       | 50 KB     | ##################################### | 100% 
psycopg2-2.8.6       | 168 KB    | ##################################### | 100% 
google-auth-oauthlib | 19 KB     | ##################################### | 100% 
numexpr-2.7.3        | 201 KB    | ##################################### | 100% 
pytz-2021.1          | 239 KB    | ##################################### | 100% 
gstreamer-1.18.4     | 2.0 MB    | ##################################### | 100% 
libuuid-2.32.1       | 28 KB     | ##################################### | 100% 
py-1.10.0            | 73 KB     | ##################################### | 100% 
jupyterlab-2.2.6     | 5.9 MB    | ##################################### | 100% 
jupyter_core-4.7.1   | 72 KB     | ##################################### | 100% 
pyopenssl-20.0.1     | 48 KB

done
The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [7]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)
map_toronto

In [8]:
import folium


map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_Toronto)  
    
map_Toronto

NameError: name 'df_toronto' is not defined

In [9]:
toronto_data = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()

(0, 5)


,Postalcode,Borough,Neighborhood,Latitude,Longitude


In [10]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7,parse_html=False).add_to(map_toronto)  
map_toronto

In [11]:
CLIENT_ID = 'XLBDRGURZVOOULUGDUC4DSJZSRE5ZI0XPB1WA5RV3YL5D1TP' # your Foursquare ID
CLIENT_SECRET = '5TW0I4L1WKC5A0H1NZHEDZD535BFN1CDQ2MRHBW1VBNFUTEK' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XLBDRGURZVOOULUGDUC4DSJZSRE5ZI0XPB1WA5RV3YL5D1TP
CLIENT_SECRET:5TW0I4L1WKC5A0H1NZHEDZD535BFN1CDQ2MRHBW1VBNFUTEK


In [12]:
CLIENT_ID = 'XLBDRGURZVOOULUGDUC4DSJZSRE5ZI0XPB1WA5RV3YL5D1TP' # your Foursquare ID
CLIENT_SECRET = '5TW0I4L1WKC5A0H1NZHEDZD535BFN1CDQ2MRHBW1VBNFUTEK' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XLBDRGURZVOOULUGDUC4DSJZSRE5ZI0XPB1WA5RV3YL5D1TP
CLIENT_SECRET:5TW0I4L1WKC5A0H1NZHEDZD535BFN1CDQ2MRHBW1VBNFUTEK


In [13]:
neighborhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

NameError: name 'df_toronto' is not defined

In [14]:
CLIENT_ID = 'DPBYY4JUY3DU20ALPSUV4ONY2K1GOJJKJ1NIHBB32XEMOVYY' # Put Your Client Id
CLIENT_SECRET = '1MV443TYEP4HUO0WDUW5NQ5W10L2Y4G05NWG11WIR3NUGC5B' # Put You Client Secret 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: Hidden')
print('CLIENT_SECRET: Hidden')

Your credentails:
CLIENT_ID: Hidden
CLIENT_SECRET: Hidden


In [15]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

NameError: name 'neighborhood_latitude' is not defined

In [16]:
results = requests.get(url).json()
results

NameError: name 'url' is not defined

# Exploring Neighbourhood in Toronto